In [1]:
import pandas as pd

In [2]:
raw = pd.read_csv('S1_prep_joint.txt', sep='\t')
raw

,count,junction_aa,day,v_call,j_call,locus
0,7000,CASSPQFTGSYEQYF,0,TRBV4-3,TRBJ2-7,beta
1,3966,CASSSPIAGQSSYEQYF,0,TRBV28,TRBJ2-7,beta
2,3221,CSASLASGTGELFF,0,TRBV20-1,TRBJ2-2,beta
3,2412,CASSYGQGNQPQHF,0,TRBV6-5,TRBJ1-5,beta
4,2292,CASSWGQGVNEQYF,0,TRBV28,TRBJ2-7,beta
...,...,...,...,...,...,...
287029,2,CSVELAFF,15,TRBV29-1,TRBJ1-1,beta
287030,2,CSVGEQFF,15,TRBV29-1,TRBJ2-1,beta
287031,2,CSATDRPI,15,TRBV29-1,TRBJ2-1,beta
287032,2,CRTNEQFF,15,TRBV20-1,TRBJ2-1,beta


In [3]:
raw0 = raw[raw.day == 0]

In [4]:
raw15 = raw[raw.day == 15]

In [5]:
raw0

,count,junction_aa,day,v_call,j_call,locus
0,7000,CASSPQFTGSYEQYF,0,TRBV4-3,TRBJ2-7,beta
1,3966,CASSSPIAGQSSYEQYF,0,TRBV28,TRBJ2-7,beta
2,3221,CSASLASGTGELFF,0,TRBV20-1,TRBJ2-2,beta
3,2412,CASSYGQGNQPQHF,0,TRBV6-5,TRBJ1-5,beta
4,2292,CASSWGQGVNEQYF,0,TRBV28,TRBJ2-7,beta
...,...,...,...,...,...,...
146433,2,CSVGPYTF,0,TRBV29-1,TRBJ1-2,beta
146434,2,CSADTQYF,0,TRBV29-1,TRBJ2-3,beta
146435,2,CSANMAFF,0,TRBV29-1,TRBJ1-1,beta
146436,2,CSGHGKYF,0,TRBV20-1,TRBJ2-7,beta


In [6]:
raw15

,count,junction_aa,day,v_call,j_call,locus
146438,3492,CASSPQFTGSYEQYF,15,TRBV4-3,TRBJ2-7,beta
146439,3461,CASSSPIAGQSSYEQYF,15,TRBV28,TRBJ2-7,beta
146440,2654,CASSYGQNYNEQFF,15,TRBV5-1,TRBJ2-1,beta
146441,2249,CSASLASGTGELFF,15,TRBV20-1,TRBJ2-2,beta
146442,1999,CASSYGQGNQPQHF,15,TRBV6-5,TRBJ1-5,beta
...,...,...,...,...,...,...
287029,2,CSVELAFF,15,TRBV29-1,TRBJ1-1,beta
287030,2,CSVGEQFF,15,TRBV29-1,TRBJ2-1,beta
287031,2,CSATDRPI,15,TRBV29-1,TRBJ2-1,beta
287032,2,CRTNEQFF,15,TRBV20-1,TRBJ2-1,beta


In [5]:
neigh = 5

In [6]:
tcremp = pd.read_csv(f'yfv_S1_tcremp_clusters_neigh_{neigh}.tsv', sep='\t', )
for idx, col in enumerate(['cdr3aa', 'v', 'j']):
    tcremp[col] = tcremp.clone_id.apply(lambda x: x.split('_')[idx].split('*')[0]) 
tcremp = tcremp.drop(columns=['clone_id']).rename(columns={'cluster_id': 'cluster'})[['cdr3aa', 'v', 'j', 'cluster']]
tcremp = tcremp[tcremp.cluster!=-1]
tcremp

,cdr3aa,v,j,cluster
3,CASSYGQGNQPQHF,TRBV6-5,TRBJ1-5,4259
10,CASSQESGETYNEQFF,TRBV4-3,TRBJ2-1,1448
18,CASSQDHAGETDTQYF,TRBV4-3,TRBJ2-3,3558
25,CASSLGRLYEQYF,TRBV27,TRBJ2-7,0
32,CASSLQGDYEQYF,TRBV7-9,TRBJ2-7,1
...,...,...,...,...
263746,CSGVGYTF,TRBV20-1,TRBJ1-2,3458
263747,CSVELAFF,TRBV29-1,TRBJ1-1,5835
263748,CSVGEQFF,TRBV29-1,TRBJ2-1,78
263749,CSATDRPI,TRBV29-1,TRBJ2-1,78


In [7]:
def mask_sequence(seq):
    return {seq[:i] + 'X' + seq[i+1:] for i in range(len(seq))}

def mask_all_sequences(sequences):
    masked_set = set()
    for seq in sequences:
        masked_set.update(mask_sequence(seq))
    return masked_set

In [8]:
def compute_usage(df_main, df_day0, df_day15):
    df_day0 = df_day0.copy()
    df_day15 = df_day15.copy()
    df_day0['usage'] = df_day0['count'] / df_day0['count'].sum()
    df_day15['usage'] = df_day15['count'] / df_day15['count'].sum()

    def get_usage(df):
        return df.groupby(['junction_aa', 'v_call', 'j_call'])['usage'].sum().reset_index().rename(
            columns={'junction_aa': 'cdr3aa', 'v_call': 'v', 'j_call': 'j'}
        )

    usage_day0 = get_usage(df_day0).rename(columns={'usage': 'day0_usage'})
    usage_day15 = get_usage(df_day15).rename(columns={'usage': 'day15_usage'})

    df = df_main.copy()
    df = df.merge(usage_day0, on=['cdr3aa', 'v', 'j'], how='left')
    df = df.merge(usage_day15, on=['cdr3aa', 'v', 'j'], how='left')

    df[['day0_usage', 'day15_usage']] = df[['day0_usage', 'day15_usage']].fillna(0)

    return df


In [9]:
vdjdb = pd.read_csv('vdjdb.slim.txt', sep='\t')

In [10]:
vdjdb = pd.read_csv('vdjdb.slim.txt', sep='\t')
vdjdb = vdjdb[(vdjdb.gene == 'TRB') & (vdjdb['antigen.species'] == 'YFV')]

In [11]:
vdjdb.cdr3

34435        CAIQDAGASYEQYF
34544    CAISDLAGEPKTQETQYF
34777       CAISESPSGALGQFF
34856       CAISEVLTGAYGYTF
35117              CANNCGFF
                ...        
84604        CSVSGERGTDTQYF
84649        CSVTGERGTDTQYF
84690       CSVVDAAPGANVLTF
84691      CSVVDAAPGGSYEQYF
84740       CSVVLVDRGADTQYF
Name: cdr3, Length: 402, dtype: object

In [12]:
def prepare_usage_table(usage_df):
    vdjdb_yfv_masks = mask_all_sequences(vdjdb.cdr3)

    usage_df['vdjdb_yfv'] = usage_df.cdr3aa.apply(lambda x: len(mask_sequence(x).intersection(vdjdb_yfv_masks)) > 0).astype(int)

    usage_df['day0_found'] = (usage_df.day0_usage > 0).astype(int)
    usage_df['day15_found'] = (usage_df.day15_usage > 0).astype(int)

    usage_df['vdjdb_yfv_day0'] = usage_df['vdjdb_yfv'] + usage_df['day0_found'] == 2
    usage_df['vdjdb_yfv_day15'] = usage_df['vdjdb_yfv'] + usage_df['day15_found'] == 2
    return usage_df

In [13]:
def prepare_cluster_stats(usage_df):
    usage_df = usage_df.copy()
    
    delta_df = usage_df[['cluster', 
                         'day0_usage', 'day0_found', 
                         'day15_usage', 'day15_found', 
                         'vdjdb_yfv', 'vdjdb_yfv_day0', 'vdjdb_yfv_day15']].groupby(
        'cluster').sum().reset_index()
    cluster_sizes = usage_df.cluster.value_counts().reset_index().rename(columns={'count': 'cluster_size' })

    delta_df['delta'] = (delta_df.day15_usage + 0.0000001) / (delta_df.day0_usage + 0.0000001)
    delta_df = delta_df.merge(cluster_sizes)
    delta_df['yfv_15_frac_in_cluster'] = delta_df.vdjdb_yfv_day15 / delta_df.cluster_size
    return delta_df

In [14]:
tcremp_usage_df = compute_usage(tcremp, raw0, raw15)
tcremp_usage = prepare_usage_table(tcremp_usage_df)
tcremp_clusters = prepare_cluster_stats(tcremp_usage)
tcremp_clusters

,cluster,day0_usage,day0_found,day15_usage,day15_found,vdjdb_yfv,vdjdb_yfv_day0,vdjdb_yfv_day15,delta,cluster_size,yfv_15_frac_in_cluster
0,0,0.001055,25,0.001439,38,13,6,10,1.363971,55,0.181818
1,1,0.001101,50,0.000714,48,7,4,4,0.648528,82,0.048780
2,2,0.000750,6,0.000527,3,0,0,0,0.702768,6,0.000000
3,3,0.001091,57,0.001174,60,21,15,11,1.076296,107,0.102804
4,4,0.000865,40,0.005605,47,0,0,0,6.483224,79,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
7207,7207,0.000000,0,0.000011,3,0,0,0,115.200010,3,0.000000
7208,7208,0.000000,0,0.000011,3,0,0,0,115.200010,3,0.000000
7209,7209,0.000013,2,0.000010,2,0,0,0,0.716093,3,0.000000
7210,7210,0.000000,0,0.000025,3,0,0,0,248.433355,3,0.000000


In [15]:
tcremp_clusters[tcremp_clusters.cluster_size > 10].sort_values(by='yfv_15_frac_in_cluster', ascending=False)

,cluster,day0_usage,day0_found,day15_usage,day15_found,vdjdb_yfv,vdjdb_yfv_day0,vdjdb_yfv_day15,delta,cluster_size,yfv_15_frac_in_cluster
2487,2487,0.000034,6,0.000038,7,10,5,7,1.110288,11,0.636364
957,957,0.000027,3,0.000263,20,12,1,12,9.819613,21,0.571429
5855,5855,0.000008,1,0.001110,12,6,0,6,143.809321,13,0.461538
2902,2902,0.000017,3,0.000084,10,5,2,5,4.864136,11,0.454545
1038,1038,0.000055,7,0.000027,5,6,2,5,0.483476,11,0.454545
...,...,...,...,...,...,...,...,...,...,...,...
994,994,0.000032,6,0.000030,6,0,0,0,0.940934,11,0.000000
999,999,0.000076,14,0.000036,6,0,0,0,0.475474,19,0.000000
227,227,0.000232,37,0.000242,48,0,0,0,1.040497,80,0.000000
1005,1005,0.000036,7,0.000025,4,0,0,0,0.684773,11,0.000000


In [16]:
tcremp_clusters.to_csv(f'tcremp_clusters_summary_{neigh}.csv', index=False)